# **Question 01**

## **Problem Statement**
You are given a **4x4 sliding tile puzzle** consisting of **15 numbered tiles (1-15)** and **one empty space (0)**.  
Your task is to implement the **A\* search algorithm** to find the **shortest sequence of moves** that transforms the given **start state** into the **goal state** using the **Manhattan distance heuristic**.

---

## **Initial and Goal States**
### **Start State (Given)**
```python
start_state = [
    [1, 2, 3, 4],
    [5, 6, 0, 8],
    [9, 10, 7, 11],
    [13, 14, 15, 12]
]

goal_state = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]
```
### **Expected Output:**

*     Sequence of moves required to reach the goal (e.g., "Move tile 7 left").
*     Final grid configuration after applying the moves.

### **Constraints:**
1.     You can only move the empty space (0) to adjacent positions (up, down, left, right).
2.     Moving LEFT or RIGHT has a cost of 1.
3.     Moving UP or DOWN has a cost of 2.
4.     The algorithm should use f(n) = g(n) + h(n) where g(n) is the total move cost.
5.     The goal is to minimize the number of moves to reach the solved state.
6.     **Heuristic Function:** The Manhattan distance between each tile’s current position and its goal position. The Manhattan distance for each tile is the sum of the horizontal and vertical distance between its current and goal positions.



In [17]:
import heapq

# Start and goal states
start_state = [
    [1, 2, 3, 4],
    [5, 6, 0, 8],
    [9, 10, 7, 11],
    [13, 14, 15, 12]
]

goal_state = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]

def manhattan_distance(state):

    """Compute the Manhattan distance heuristic for a given puzzle state."""

    distance = 0
    for i in range(4):
        for j in range(4):
            val = state[i][j]
            if val != 0:
                goal_row = (val - 1) // 4
                goal_col = (val - 1) % 4
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance

def get_neighbors(state):
    """Generate all valid moves by shifting the empty space (0)."""
    neighbors = []
    moves = {'Up': (-1, 0, 2), 'Down': (1, 0, 2), 'Left': (0, -1, 1), 'Right': (0, 1, 1)}

    # Find empty tile (0)
    x, y = next((i, j) for i in range(4) for j in range(4) if state[i][j] == 0)

    for move, (dx, dy, cost) in moves.items():
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < 4 and 0 <= new_y < 4:
            new_state = [row[:] for row in state]
            # Swap 0 with adjacent tile
            new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
            neighbors.append((new_state, move, new_state[x][y], cost))

    return neighbors


In [18]:
import heapq


def a_star_search(start, goal):
    """Implement A* search algorithm to find the shortest path from start to goal state."""
    frontier = []
    heapq.heappush(frontier, (manhattan_distance(start), 0, start, []))
    visited = set()

    while frontier:
        f, g, current, path = heapq.heappop(frontier)
        state_tuple = tuple(tuple(row) for row in current)

        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        if current == goal:
            return path, current

        for neighbor, move, tile, cost in get_neighbors(current):
            new_g = g + cost
            new_f = new_g + manhattan_distance(neighbor)
            heapq.heappush(frontier, (new_f, new_g, neighbor, path + [f"Move tile {tile} {move}"]))

    return None, None


In [19]:

# Run the A* search
solution_moves, final_state = a_star_search(start_state, goal_state)

# Output the results
if solution_moves:
    print("Solution Moves:")
    for move in solution_moves:
        print(move)
    print("\nFinal Configuration:")
    for row in final_state:
        print(row)
else:
    print("No solution found.")


Solution Moves:
Move tile 7 Down
Move tile 11 Right
Move tile 12 Down

Final Configuration:
[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0]


# **Question 02**

A company is launching a new product and needs to decide the best pricing strategy to maximize profit. The company can choose from different price points, but each price point affects both demand (units sold) and profit per unit. You are given a set of possible price points, along with the expected demand (units sold at that price) and profit per unit. Your task is to use Steepest-Ascent Hill Climbing to find the best price point that maximizes overall profit. After implementing Steepest-Ascent Hill Climbing, modify your approach to apply Random Restart Hill Climbing, running the steepest-ascent algorithm multiple times from different starting points to improve the solution.

Implement the Hill Climbing Algorithm to find the best price point using the following steps:

*   Start with a random price point from the given options.
*   Iterate until a local maximum is reached.
*   Evaluate neighboring price points and select the one that increases the total profit the most.
*   Update the current price point with the selected neighbor.
*   Update the best price point if the new state gives a higher profit.

In [20]:
import random

# Price points and their corresponding demand and profit per unit
pricing_options = {
    "price1": {"profit_per_unit": 5, "demand": 500},
    "price2": {"profit_per_unit": 8, "demand": 400},
    "price3": {"profit_per_unit": 12, "demand": 300},
    "price4": {"profit_per_unit": 15, "demand": 200},
    "price5": {"profit_per_unit": 20, "demand": 150},
    "price6": {"profit_per_unit": 25, "demand": 100}
}


In [21]:
###  **Evaluation Function**
def evaluate(state):
    """
    Compute total profit for the given price point.
    """
    profit_per_unit = pricing_options[state]["profit_per_unit"]
    demand = pricing_options[state]["demand"]
    return profit_per_unit * demand


In [22]:
### **Steepest-Ascent Hill Climbing Algorithm**
def hill_climbing(max_iter=50):
    """
    Steepest-Ascent Hill Climbing:
    - Start with a random price point.
    - Move to the neighbor with the highest profit gain until no improvement is possible.
    """
    price_keys = list(pricing_options.keys())
    current_index = random.randint(0, len(price_keys) - 1)
    current_state = price_keys[current_index]
    current_score = evaluate(current_state)
    path = [current_state]

    for _ in range(max_iter):
        neighbors = []
        # Check neighbors: previous and next price points in the list
        if current_index - 1 >= 0:
            neighbors.append(current_index - 1)
        if current_index + 1 < len(price_keys):
            neighbors.append(current_index + 1)

        # Find the best neighbor
        best_neighbor_index = current_index
        best_score = current_score
        for neighbor_index in neighbors:
            neighbor_state = price_keys[neighbor_index]
            neighbor_score = evaluate(neighbor_state)
            if neighbor_score > best_score:
                best_score = neighbor_score
                best_neighbor_index = neighbor_index

        # If no neighbor improves the score, stop (local maximum found)
        if best_neighbor_index == current_index:
            break
        else:
            current_index = best_neighbor_index
            current_state = price_keys[current_index]
            current_score = best_score
            path.append(current_state)

    return current_state, current_score, path


In [23]:

###  **Random Restart Hill Climbing Algorithm**
def random_restart_hill_climbing(restarts=5, max_iter=50):
    """
    - Run Steepest-Ascent Hill Climbing multiple times with random starts.
    - Return the best overall price point found.
    """
    best_overall_state = None
    best_overall_score = float('-inf')
    best_overall_path = []

    for i in range(restarts):
        state, score, path = hill_climbing(max_iter=max_iter)
        print(f"Restart {i+1}: Best Price = {state}, Profit = {score}")
        if score > best_overall_score:
            best_overall_score = score
            best_overall_state = state
            best_overall_path = path

    return best_overall_state, best_overall_score, best_overall_path

In [24]:
# **Run Steepest-Ascent Hill Climbing**
best_state, best_score, path = hill_climbing()
print("\n--- Steepest-Ascent Hill Climbing ---")
print(f"Best price point: {best_state}")
print(f"Maximum total profit: {best_score}")
print(f"Path to solution: {path}")

# **Run Random Restart Hill Climbing**
best_state_rr, best_score_rr, path_rr = random_restart_hill_climbing(restarts=5)
print("\n--- Random Restart Hill Climbing ---")
print(f"Best price point after random restarts: {best_state_rr}")
print(f"Maximum total profit: {best_score_rr}")
print(f"Path to solution: {path_rr}")


--- Steepest-Ascent Hill Climbing ---
Best price point: price3
Maximum total profit: 3600
Path to solution: ['price4', 'price3']
Restart 1: Best Price = price5, Profit = 3000
Restart 2: Best Price = price3, Profit = 3600
Restart 3: Best Price = price3, Profit = 3600
Restart 4: Best Price = price3, Profit = 3600
Restart 5: Best Price = price5, Profit = 3000

--- Random Restart Hill Climbing ---
Best price point after random restarts: price3
Maximum total profit: 3600
Path to solution: ['price3']
